In [1]:
%pip install biodivine_aeon==1.0.0a8

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 21.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
from biodivine_aeon import *
from pathlib import Path

Detected IPython (`ZMQInteractiveShell`). Log level set to `LOG_ESSENTIAL`.


In [3]:
model = BooleanNetwork.from_file("model_map1.aeon")
print(model)

BooleanNetwork(variables=91, regulations=124, explicit_parameters=37, implicit_parameters=22)


In [4]:
stg = AsynchronousGraph(model)

First, we identify the unknown functions that can be safely eliminated because they have only one interpretations that satisfies all the requirements regarding their input monotonicity and essentiality.

In [5]:
to_eliminate = []
for param in model.explicit_parameters():
    print(model.get_explicit_parameter_name(param))
    count = 0
    for m in stg.mk_unit_colors().items(retained=[param]):
        print(" > ", m)
        count += 1
        if count == 100:
            break
    if count == 1:
        to_eliminate.append(param)

cat_LAT2_phosphorylated_palmytoylated_47
 >  ColorModel({'cat_LAT2_phosphorylated_palmytoylated_47': '(x_0 & x_1)'})
 >  ColorModel({'cat_LAT2_phosphorylated_palmytoylated_47': '((!x_0 & x_1) | x_0)'})
cat_PIP3_simple_molecule_55
 >  ColorModel({'cat_PIP3_simple_molecule_55': 'x_0'})
cat_SHIP_1_phosphorylated_52
 >  ColorModel({'cat_SHIP_1_phosphorylated_52': 'x_0'})
cat_FceRI_phosphorylated_phosphorylated_46
 >  ColorModel({'cat_FceRI_phosphorylated_phosphorylated_46': 'x_0'})
cat_DAG_simple_molecule_65
 >  ColorModel({'cat_DAG_simple_molecule_65': '(x_0 & x_1)'})
 >  ColorModel({'cat_DAG_simple_molecule_65': '((!x_0 & x_1) | x_0)'})
cat_FceRI_Syk_complex_Cytosol_1_25
 >  ColorModel({'cat_FceRI_Syk_complex_Cytosol_1_25': 'x_0'})
cat_Fyn_phosphorylated_phosphorylated_palmytoylated_62
 >  ColorModel({'cat_Fyn_phosphorylated_phosphorylated_palmytoylated_62': 'x_0'})
cat_MEKK1_MAP3K1__phosphorylated_74
 >  ColorModel({'cat_MEKK1_MAP3K1__phosphorylated_74': 'x_0'})
cat_LAT1_phosphorylated_

In [6]:
print(to_eliminate)

[ParameterId(30), ParameterId(32), ParameterId(13), ParameterId(12), ParameterId(15), ParameterId(26), ParameterId(21), ParameterId(3), ParameterId(31), ParameterId(1), ParameterId(25), ParameterId(6), ParameterId(4), ParameterId(19), ParameterId(14), ParameterId(23), ParameterId(28), ParameterId(24), ParameterId(33), ParameterId(17), ParameterId(10), ParameterId(36), ParameterId(2), ParameterId(0), ParameterId(27), ParameterId(16)]


In [7]:
m = next(stg.mk_unit_colors().items(retained = to_eliminate))

In [8]:
model_simplified = m.instantiate(model)

In [9]:
Path("model_map1.simplified.aeon").write_text(model_simplified.to_aeon())

13664

In [10]:
stg = AsynchronousGraph(model_simplified)

In [11]:
# This identifies model outputs. In the next step, we restrict them manually to
# a subset of outputs that are biologically interpretable.
outputs = []
for var in model_simplified.variable_names():
    if len(model_simplified.successors(var)) == 0:
        outputs.append(var)
outputs

['Ca2_slash_DAG_RASGRP1_RAS_C_RAF_complex',
 'Elk1_phosphorylated',
 'Fyn_phosphorylated_phosphorylated_palmytoylated',
 'I_kB_phosphorylated',
 'Jun_Fos_complex',
 'LAT2_Grb2_Shc_SOS_H_RAS_RAF1_complex',
 'NFAT_nucleus',
 'NF_kappa_B_nucleus',
 'PIP3_Akt_complex',
 'c_JUN_phosphorylated_nucleus']

In [12]:
# Actually relevant outputs:
outputs = ['Ca2_slash_DAG_RASGRP1_RAS_C_RAF_complex',
 'Elk1_phosphorylated',
 'Jun_Fos_complex',
 'NFAT_nucleus',
 'NF_kappa_B_nucleus',
 'PIP3_Akt_complex']
outputs

['Ca2_slash_DAG_RASGRP1_RAS_C_RAF_complex',
 'Elk1_phosphorylated',
 'Jun_Fos_complex',
 'NFAT_nucleus',
 'NF_kappa_B_nucleus',
 'PIP3_Akt_complex']

In [13]:
inputs = []
for var in model_simplified.variable_names():
    if len(model_simplified.predecessors(var)) == 0:
        inputs.append(var)
inputs

['Akt',
 'Bcr',
 'Btk',
 'CRAC',
 'Csk',
 'Dok1_RasGAP_complex',
 'Fyn_phosphorylated_palmytoylated',
 'GRAP2',
 'H_RAS_Plasma_space_Membrane',
 'LAT2_Grb2_complex',
 'PAK2',
 'PIP2_simple_molecule',
 'PI_3K',
 'PKC_space_Theta',
 'PLCG1',
 'RAF1_Cytosol',
 'RASGRP1',
 'Rac1',
 'Shc_Grb2_complex',
 'Syk',
 'Vav1',
 'cCbl']

In [14]:
params = []
for p in model_simplified.explicit_parameter_names():
    params.append((p, model_simplified.get_explicit_parameter_arity(p)))
params

[('cat_Calmodulin_Calcineurin_complex_19', 2),
 ('cat_DAG_simple_molecule_65', 2),
 ('cat_Elk1_phosphorylated_53', 3),
 ('cat_Erk_MAPK1_3__phosphorylated_49', 2),
 ('cat_FceRI_Syk_complex_Cytosol_1_24', 1),
 ('cat_IP3_simple_molecule_57', 2),
 ('cat_JNK_phosphorylated_70', 2),
 ('cat_LAT2_phosphorylated_palmytoylated_47', 2),
 ('cat_PIP3_Akt_complex_8', 1),
 ('cat_SOS_Grb2_complex_5', 1),
 ('cat_Shc_phosphorylated_56', 2)]

In [15]:
ctx = SymbolicSpaceContext(model_simplified)
stg = AsynchronousGraph(model_simplified, ctx)

Compute minimal trap spaces across all interpretations. These should almost exactly correspond to model attractors (this can take a few minutes).

In [16]:
traps = TrapSpaces.essential_symbolic(ctx, stg, ctx.mk_unit_colored_spaces(stg))

Start symbolic essential trap space search with 2024261176623274600000000000000000000000000000000000000[nodes:236] candidates.
 > Merge. New result has 123457 BDD nodes. Remaining constraints: 10.
 > Merge. New result has 152944 BDD nodes. Remaining constraints: 9.
 > Merge. New result has 158220 BDD nodes. Remaining constraints: 8.
 > Merge. New result has 218609 BDD nodes. Remaining constraints: 7.
 > Merge. New result has 195497 BDD nodes. Remaining constraints: 6.
 > Merge. New result has 209627 BDD nodes. Remaining constraints: 5.
 > Merge. New result has 257497 BDD nodes. Remaining constraints: 4.
 > Merge. New result has 248221 BDD nodes. Remaining constraints: 3.
 > Merge. New result has 314759 BDD nodes. Remaining constraints: 2.
 > Merge. New result has 429900 BDD nodes. Remaining constraints: 1.
 > Merge. New result has 884192 BDD nodes. Remaining constraints: 0.
Merge finished with 884192 BDD nodes.
Found 77309411328x71795760[nodes:884192] essential trap spaces.


In [17]:
min_traps = TrapSpaces.minimize(ctx, traps)

Start minimal subspace search with 77309411328x71795760[nodes:884192] candidates.
Computing super-spaces: 382747566778465550000000000000000000000[nodes:580482].
Computing super-spaces: 765425037549340900000000000000000000000[nodes:580480].
Computing super-spaces: 1530850075098681800000000000000000000000[nodes:580476].
Computing super-spaces: 3061700150197363500000000000000000000000[nodes:580473].
Computing super-spaces: 6123400300394727000000000000000000000000[nodes:580467].
Computing super-spaces: 12246800600789454000000000000000000000000[nodes:580462].
Computing super-spaces: 24493601201578910000000000000000000000000[nodes:580446].
Computing super-spaces: 48958042464000285000000000000000000000000[nodes:580419].
Computing super-spaces: 97886924988843040000000000000000000000000[nodes:580369].
Computing super-spaces: 195773849977686080000000000000000000000000[nodes:580303].
Computing super-spaces: 391547699955372150000000000000000000000000[nodes:580237].
Computing super-spaces: 78309539

In [18]:
min_traps

ing super-spaces: 55089026982265610000000000000000000000000000000000000000[nodes:1457856].
Computing super-spaces: 110178053964531230000000000000000000000000000000000000000[nodes:1454968].
Computing super-spaces: 220356107929062460000000000000000000000000000000000000000[nodes:1456828].
Computing super-spaces: 440712215858124900000000000000000000000000000000000000000[nodes:1454952].
Computing super-spaces: 881424431716249800000000000000000000000000000000000000000[nodes:1454756].
Computing super-spaces: 1762848863432499700000000000000000000000000000000000000000[nodes:1452228].
Computing super-spaces: 3525697726864999000000000000000000000000000000000000000000[nodes:1453608].
Computing super-spaces: 7051395280569585000000000000000000000000000000000000000000[nodes:1451980].
Computing super-spaces: 14102790561139170000000000000000000000000000000000000000000[nodes:1449920].
Computing super-spaces: 28205581122278340000000000000000000000000000000000000000000[nodes:1447860].
Computing super-spac

ColoredSpaceSet(cardinality=228493099008, colors=77309411328, spaces=35929632, symbolic_size=588576)

mputing super-spaces: 57765029290263640000000000000000000000000000000000000000000000[nodes:11011833].
Computing super-spaces: 115530058580527270000000000000000000000000000000000000000000000[nodes:13906630].
Computing super-spaces: 231060117161054550000000000000000000000000000000000000000000000[nodes:13945314].
Computing super-spaces: 462120234322109100000000000000000000000000000000000000000000000[nodes:13945278].
Computing super-spaces: 924240468644218200000000000000000000000000000000000000000000000[nodes:13945242].
Computing super-spaces: 1848480937242817800000000000000000000000000000000000000000000000[nodes:14634694].
Computing super-spaces: 3696961874440016000000000000000000000000000000000000000000000000[nodes:17403880].
Computing super-spaces: 7393923748834413000000000000000000000000000000000000000000000000[nodes:19574168].
Computing super-spaces: 14787847497668826000000000000000000000000000000000000000000000000[nodes:19574148].
Computing super-spaces: 29575694995337653000000000000

Compute the number of output value combinations that can actually appear in the model, and the distribution of output values within these combinations.

In [19]:
phenotype_iterator = min_traps.spaces().items(retained = outputs)
phenotype_count = sum(1 for _ in phenotype_iterator)
print(f"Phenotypes: {phenotype_count}")

27398].
Computing super-spaces: 14140653146958190000000000000000000000000000000000000000000000000[nodes:3627388].
Computing super-spaces: 28281306293916380000000000000000000000000000000000000000000000000[nodes:3627388].
Computing super-spaces: 56562612587832760000000000000000000000000000000000000000000000000[nodes:3993994].
Computing super-spaces: 113125225175665520000000000000000000000000000000000000000000000000[nodes:3993998].
Computing super-spaces: 226250450351331040000000000000000000000000000000000000000000000000[nodes:3994000].
Computing super-spaces: 452500900702662100000000000000000000000000000000000000000000000000[nodes:3994001].
Minimization in progress: 0x0[nodes:1] unprocessed, 77309411328x35929632[nodes:588576] candidates.
Found 228493099008[nodes:588576] minimal spaces.
Phenotypes: 84


In [20]:
is_zero = { var: 0 for var in outputs }
is_one = { var: 0 for var in outputs }
is_any = { var: 0 for var in outputs }
for phenotype in min_traps.spaces().items(retained = outputs):
    phenotype_traps = min_traps.intersect_spaces(phenotype.to_symbolic())
    for (var, value) in phenotype.items():        
        var = ctx.get_network_variable_name(var)
        if value == True:
            is_one[var] += 1
        elif value == False:
            is_zero[var] += 1
        else:
            is_any[var] += 1

print("Zero:", is_zero)
print("One:", is_one)
print("Any:", is_any)

Zero: {'Ca2_slash_DAG_RASGRP1_RAS_C_RAF_complex': 52, 'Elk1_phosphorylated': 20, 'Jun_Fos_complex': 52, 'NFAT_nucleus': 26, 'NF_kappa_B_nucleus': 36, 'PIP3_Akt_complex': 42}
One: {'Ca2_slash_DAG_RASGRP1_RAS_C_RAF_complex': 20, 'Elk1_phosphorylated': 24, 'Jun_Fos_complex': 12, 'NFAT_nucleus': 58, 'NF_kappa_B_nucleus': 30, 'PIP3_Akt_complex': 42}
Any: {'Ca2_slash_DAG_RASGRP1_RAS_C_RAF_complex': 12, 'Elk1_phosphorylated': 40, 'Jun_Fos_complex': 20, 'NFAT_nucleus': 0, 'NF_kappa_B_nucleus': 18, 'PIP3_Akt_complex': 0}


In [21]:
def cardinality_without_inputs(ctx, inputs, set):
    # Project the BDD to remove inputs.
    set_bdd = set.to_bdd()
    ignore_vars = [ ctx.get_function_table(i)[0][1] for i in inputs ]
    set_bdd = set_bdd.r_exists(ignore_vars)

    all_vars = ctx.bdd_variable_set().variable_count()
    function_vars = sum(len(x) for x in ctx.explicit_functions_bdd_variables().values())

    general_cardinality = set_bdd.cardinality()
    return int(general_cardinality / (2 ** (all_vars - function_vars)))

def prune_inputs(ctx, inputs, set):
    set_bdd = set.to_bdd()
    ignore_vars = [ ctx.get_function_table(i)[0][1] for i in inputs ]
    set_bdd = set_bdd.r_exists(ignore_vars)

    return ColorSet(ctx, set_bdd)

def prune_inputs_forall(ctx, inputs, set):
    set_bdd = set.to_bdd()
    ignore_vars = [ ctx.get_function_table(i)[0][1] for i in inputs ]
    set_bdd = set_bdd.r_for_all(ignore_vars)

    return ColorSet(ctx, set_bdd)

Build a table showing the multiplicity of various attractor combinations for each output variable:

In [22]:
print(f"0\t1\t*\t0-1\t1-*\t0-*\t0-1-*\tname")

for var in outputs:
    p_var = ctx.get_positive_space_variable(var)
    n_var = ctx.get_negative_space_variable(var)
    var_is_one = SpaceSet(ctx, ctx.bdd_variable_set().mk_conjunctive_clause({ p_var: True, n_var: False }))
    var_is_zero = SpaceSet(ctx, ctx.bdd_variable_set().mk_conjunctive_clause({ p_var: False, n_var: True }))
    var_is_any = SpaceSet(ctx, ctx.bdd_variable_set().mk_conjunctive_clause({ p_var: True, n_var: True }))

    one_colors = min_traps.intersect_spaces(var_is_one).colors()
    zero_colors = min_traps.intersect_spaces(var_is_zero).colors()
    any_colors = min_traps.intersect_spaces(var_is_any).colors()

    # one_colors = prune_inputs(ctx, inputs, one_colors)
    # zero_colors = prune_inputs(ctx, inputs, zero_colors)
    # any_colors = prune_inputs(ctx, inputs, any_colors)

    one = one_colors.minus(zero_colors).minus(any_colors)
    zero = zero_colors.minus(one_colors).minus(any_colors)
    any = any_colors.minus(zero_colors).minus(one_colors)
    zero_one = one_colors.intersect(zero_colors).minus(any_colors)
    one_any = one_colors.intersect(any_colors).minus(zero_colors)
    zero_any = zero_colors.intersect(any_colors).minus(one_colors)
    zero_one_any = zero_colors.intersect(one_colors).intersect(any_colors)
    
    # c_one = cardinality_without_inputs(ctx, inputs, one)
    # c_zero = cardinality_without_inputs(ctx, inputs, zero)
    # c_any = cardinality_without_inputs(ctx, inputs, any)
    # c_zero_one = cardinality_without_inputs(ctx, inputs, zero_one)
    # c_one_any = cardinality_without_inputs(ctx, inputs, one_any)
    # c_zero_any = cardinality_without_inputs(ctx, inputs, zero_any)
    # c_zero_one_any = cardinality_without_inputs(ctx, inputs, zero_one_any)

    c_one = one.cardinality()
    c_zero = zero.cardinality()
    c_any = any.cardinality()
    c_zero_one = zero_one.cardinality()
    c_one_any = one_any.cardinality()
    c_zero_any = zero_any.cardinality()
    c_zero_one_any = zero_one_any.cardinality()

    print(f"{c_zero}\t{c_one}\t{c_any}\t{c_zero_one}\t{c_one_any}\t{c_zero_any}\t{c_zero_one_any}\t{var}")    

0	1	*	0-1	1-*	0-*	0-1-*	name
76880019456	207618048	7077888	207618048	0	7077888	0	Ca2_slash_DAG_RASGRP1_RAS_C_RAF_complex
76910952448	351272960	47185920	0	0	0	0	Elk1_phosphorylated
77224476672	56623104	28311552	0	0	0	0	Jun_Fos_complex
0	38654705664	0	38654705664	0	0	0	NFAT_nucleus
75591843840	1703411712	14155776	0	0	0	0	NF_kappa_B_nucleus
75723964416	1585446912	0	0	0	0	0	PIP3_Akt_complex


Finally, let us compare the partial network to the original CaSQ network:

In [23]:
model_known = BooleanNetwork.from_file("model_map1.full.aeon")
print(model_known)

BooleanNetwork(variables=91, regulations=124, explicit_parameters=0, implicit_parameters=22)


In [24]:
ctx_known = SymbolicSpaceContext(model_known)
stg_known = AsynchronousGraph(model_known, ctx_known)

In [25]:
traps_known = TrapSpaces.essential_symbolic(ctx_known, stg_known, ctx_known.mk_unit_colored_spaces(stg_known))

Start symbolic essential trap space search with 109823197516453700000000000000000000000000000000000[nodes:184] candidates.
 > Merge. New result has 109532 BDD nodes. Remaining constraints: 3.
 > Merge. New result has 130676 BDD nodes. Remaining constraints: 2.
 > Merge. New result has 152921 BDD nodes. Remaining constraints: 1.
 > Merge. New result has 317507 BDD nodes. Remaining constraints: 0.
Merge finished with 317507 BDD nodes.
Found 4194304x24182784[nodes:317507] essential trap spaces.


In [26]:
min_traps_known = TrapSpaces.minimize(ctx_known, traps_known)

Start minimal subspace search with 4194304x24182784[nodes:317507] candidates.
Computing super-spaces: 20766652132938854000000000000000000[nodes:213005].
Computing super-spaces: 41530768964677250000000000000000000[nodes:213003].
Computing super-spaces: 83061537929354500000000000000000000[nodes:212999].
Computing super-spaces: 166123075858709000000000000000000000[nodes:212996].
Computing super-spaces: 332246151717418000000000000000000000[nodes:212990].
Computing super-spaces: 664492303434836000000000000000000000[nodes:212983].
Computing super-spaces: 1328984606869672000000000000000000000[nodes:212971].
Computing super-spaces: 2656914528439954200000000000000000000[nodes:212949].
Computing super-spaces: 5312774371580518600000000000000000000[nodes:212919].
Computing super-spaces: 10625548743161037000000000000000000000[nodes:212885].
Computing super-spaces: 21251097486322074000000000000000000000[nodes:212851].
Computing super-spaces: 42502194972644150000000000000000000000[nodes:212833].
Comp

In [27]:
min_traps_transferred = ColoredSpaceSet(ctx, ctx.transfer_from(min_traps_known.to_bdd(), ctx_known))

1235778].
Computing super-spaces: 12541943564952172000000000000000000000000000000000000000000[nodes:1240756].
Computing super-spaces: 25083887129904344000000000000000000000000000000000000000000[nodes:1240728].
Computing super-spaces: 50167774259808690000000000000000000000000000000000000000000[nodes:1608392].
Computing super-spaces: 100335548519617370000000000000000000000000000000000000000000[nodes:1842320].
Computing super-spaces: 200671097039234750000000000000000000000000000000000000000000[nodes:2504548].
Computing super-spaces: 401342194078469500000000000000000000000000000000000000000000[nodes:3275628].
Computing super-spaces: 802684388156939000000000000000000000000000000000000000000000[nodes:4005542].
Computing super-spaces: 1605368776313878000000000000000000000000000000000000000000000[nodes:4005550].
Computing super-spaces: 3210737552627756000000000000000000000000000000000000000000000[nodes:4232040].
Computing super-spaces: 6421475105255512000000000000000000000000000000000000000000

In [28]:
equivalent = prune_inputs_forall(ctx, inputs, min_traps.intersect(min_traps_transferred).colors())
print("All models:", cardinality_without_inputs(ctx, inputs, stg.mk_unit_colors()))
print("Equivalent models:", cardinality_without_inputs(ctx, inputs, equivalent))

All models: 18432
Equivalent models: 128


Let's examine which funtions are fixed/free in the 128 equivalent models:

In [29]:
for param in model_simplified.explicit_parameters():
    print(model_simplified.get_explicit_parameter_name(param))
    count = 0
    for m in equivalent.items(retained=[param]):
        print(" > ", m)
        count += 1
        if count == 100:
            break
    total = 0
    for m in stg.mk_unit_colors().items(retained=[param]):
        total += 1
    print(f"{count}/{total}")

cat_Elk1_phosphorylated_53
 >  ColorModel({'cat_Elk1_phosphorylated_53': '((!x_0 & (x_1 & x_2)) | x_0)'})
 >  ColorModel({'cat_Elk1_phosphorylated_53': '((!x_0 & (!x_1 & x_2)) | ((!x_0 & x_1) | x_0))'})
2/9
cat_Erk_MAPK1_3__phosphorylated_49
 >  ColorModel({'cat_Erk_MAPK1_3__phosphorylated_49': '(x_0 & x_1)'})
 >  ColorModel({'cat_Erk_MAPK1_3__phosphorylated_49': '((!x_0 & x_1) | x_0)'})
2/2
cat_IP3_simple_molecule_57
 >  ColorModel({'cat_IP3_simple_molecule_57': '(x_0 & x_1)'})
 >  ColorModel({'cat_IP3_simple_molecule_57': '((!x_0 & x_1) | x_0)'})
2/2
cat_PIP3_Akt_complex_8
 >  ColorModel({'cat_PIP3_Akt_complex_8': 'x_0'})
1/2
cat_SOS_Grb2_complex_5
 >  ColorModel({'cat_SOS_Grb2_complex_5': 'x_0'})
1/2
cat_FceRI_Syk_complex_Cytosol_1_24
 >  ColorModel({'cat_FceRI_Syk_complex_Cytosol_1_24': 'x_0'})
 >  ColorModel({'cat_FceRI_Syk_complex_Cytosol_1_24': 'true'})
2/2
cat_Calmodulin_Calcineurin_complex_19
 >  ColorModel({'cat_Calmodulin_Calcineurin_complex_19': '(x_0 & x_1)'})
 >  ColorMod